# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [13]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [ ]:
# Use this cell for any import statements you add



---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [14]:
artists = {'robyn':"https://www.azlyrics.com/r/robyn.html",
           'cher':"https://www.azlyrics.com/c/cher.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: After looking over the robots.txt file for the website we are allowed teh scrap song lyric data from here. However it is important to note that we are not allowed to scrap data that has /lyricsdb/ or /song/ in the url and since we are building custom URLs with the artist name and song name we are allowed to scrap data from this website.


NEED CELL BELOW

In [16]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)
for artist, artist_page in artists.items() :
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    
    # Test for good request and soupify
    if r.status_code == 200 :
        soup = BeautifulSoup(r.text,"html.parser")
            
        # many ways to do this, but here's a quick way to get 
        # a tags with href values matching the lyrics pages
        for page_link in soup.find_all('a',href=re.compile("/lyrics/")) :
            lyrics_pages[artist].append(page_link.attrs['href'])

In [17]:
base_url = "https://www.azlyrics.com"
for artist, urls in lyrics_pages.items():
    for url in urls:
        print(f"{base_url}{url}")
#lyrics_pages.get('cher')

https://www.azlyrics.com/lyrics/robyn/bumpyride.html
https://www.azlyrics.com/lyrics/robyn/inmyheart.html
https://www.azlyrics.com/lyrics/robyn/youvegotthatsomething.html
https://www.azlyrics.com/lyrics/robyn/doyouknowwhatittakes.html
https://www.azlyrics.com/lyrics/robyn/thelasttime.html
https://www.azlyrics.com/lyrics/robyn/justanothergirlfriend.html
https://www.azlyrics.com/lyrics/robyn/dontwantyouback.html
https://www.azlyrics.com/lyrics/robyn/doyoureallywantmeshowrespect.html
https://www.azlyrics.com/lyrics/robyn/how.html
https://www.azlyrics.com/lyrics/robyn/herewego.html
https://www.azlyrics.com/lyrics/robyn/wheredidourlovego.html
https://www.azlyrics.com/lyrics/robyn/robynishere.html
https://www.azlyrics.com/lyrics/robyn/iwish.html
https://www.azlyrics.com/lyrics/robyn/showmelove.html
https://www.azlyrics.com/lyrics/robyn/play.html
https://www.azlyrics.com/lyrics/robyn/myonlyreason.html
https://www.azlyrics.com/lyrics/robyn/underneaththeheart.html
https://www.azlyrics.com/lyric

Let's make sure we have enough lyrics pages to scrape. 

## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [18]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [6]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.
import shutil
if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [23]:
def fetch_and_save_lyrics(lyrics_pages):
    base_path = 'lyrics'
    base_url = "https://www.azlyrics.com"
    os.makedirs(base_path, exist_ok=True)  # Step 1: Create 'lyrics' folder

    for artist, urls in lyrics_pages.items():
        artist_path = os.path.join(base_path, artist.replace(" ", "_").lower())  # Normalize the artist name
        os.makedirs(artist_path, exist_ok=True)  # Step 2: Create subfolder for each artist

        for url in urls:
            try:
                response = requests.get(f"{base_url}{url}")
                time.sleep(5 + 10*random.random())
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    lyrics_div = soup.find('div', class_=False, id=False)  # Adjust selector as needed
                    lyrics = lyrics_div.get_text(strip=True) if lyrics_div else "Lyrics not found."

                    filename = generate_filename_from_link(url)
                    if filename:
                        file_path = os.path.join(artist_path, filename)
                        with open(file_path, 'w', encoding='utf-8') as file:
                            file.write(lyrics)  # Step 5: Write lyrics to file
            except Exception as e:
                print(f"Failed to process {url}: {str(e)}")
    

In [28]:
## The cell below calls the function and it works but it goes on and on not sure but 
##it loaded data into the folders

In [27]:
fetch_and_save_lyrics(lyrics_pages)

In [ ]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [25]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [26]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For robyn we have 104 files.
For robyn we have roughly 27249 words, 3509 are unique.
For cher we have 173 files.
For cher we have roughly 30470 words, 5345 are unique.
